In [1]:
import pandas as pd
import duckdb

In [2]:
# Connect to Database
con = duckdb.connect(database='database.db', read_only=False)

In [3]:
# Mac Path
# nets_pub_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_companies_public_maris.dta')
# nets_all_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_all_maris.dta')
# compustat_all = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat.dta')
# Workstation Path
nets_all_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_all_maris.dta')
nets_pub_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_companies_public_maris.dta')
nets_city_state = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/nets_city_state.dta')
compustat_all = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat.dta')

# ['HQDuns', 'HQCompany', 'HQAdress','HQZipCode', 'HQCity', 'HQState', 'HQArea', 'HQPhone', 'Latitude', 'Longitude', 'LevelCode']

In [ ]:
# compustat_geocoded = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat_geocoded_final.dta')
compustat_geocoded = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat_geocoded_final.dta')
# ['gvkey', 'hq_address_compustat', 'count', 'lat', 'lon']

In [7]:
con.execute('DROP TABLE IF EXISTS compustat_geocoded')
con.execute('CREATE TABLE compustat_geocoded AS SELECT * FROM compustat_geocoded')

┌──────────────┐
│ count(gvkey) │
│    int64     │
├──────────────┤
│        42980 │
└──────────────┘

-- START OF COMPUSTAT PREPROCESSING --

In [8]:
con.execute('DROP TABLE IF EXISTS compustat')
con.execute('CREATE TABLE compustat AS SELECT DISTINCT gvkey, hqcompany, hqzipcode, city FROM compustat_all')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        42979 │
└──────────────┘

In [ ]:
# """
# Extract entries of compustat_geocoded that are not in all_data_cleaned
# """
# query_str = """
# SELECT compustat_geocoded.gvkey
# FROM compustat_geocoded
# LEFT JOIN compustat ON compustat_geocoded.gvkey = compustat.gvkey
# WHERE compustat.gvkey IS NULL;
# """
# con.sql(query_str).fetchdf()

In [11]:
"""
In this cell, we want to join the geo date from compustat_geocoded with the other data from compustat_all
Also, we want to trim and upper case the city names to make the join more robust
"""

con.execute('ALTER TABLE compustat ADD COLUMN lat DECIMAL(9,6)')
con.execute('ALTER TABLE compustat ADD COLUMN lon DECIMAL(9,6)')

# TODO: Note that lat and lon are switched in compustat_geocoded. This is probably a mistake in the geocoding process
sql_query = """
UPDATE compustat
SET lat = compustat_geocoded.lon,
    lon = compustat_geocoded.lat
FROM compustat_geocoded
WHERE compustat.gvkey = compustat_geocoded.gvkey;
"""
con.execute(sql_query)
con.sql('UPDATE compustat SET city = UPPER(TRIM(city));')

In [12]:
"""
Apparently, there are 3197 ETFS in the compustat dataset. We want to get rid of them.
# """
# sql_query = """
# SELECT *
# FROM compustat
# WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
# """
# con.execute(sql_query)
"""
We delete all entries that contain the substring 'ETF'
"""
sql_query = """
DELETE FROM compustat
WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
"""
con.execute(sql_query)

┌────────┬──────────────────────────────┬────────────┬───────────────┬──────────────┬──────────────┐
│ gvkey  │          hqcompany           │ hqzipcode  │     city      │     lat      │     lon      │
│ int32  │           varchar            │  varchar   │    varchar    │ decimal(9,6) │ decimal(9,6) │
├────────┼──────────────────────────────┼────────────┼───────────────┼──────────────┼──────────────┤
│  16709 │ PROSHARES MERGER ETF         │ 20814      │ BETHESDA      │    38.982081 │   -77.093941 │
│  17051 │ GLOBAL X JUNIOR MLP ETF      │ 10158      │ NEW YORK      │    40.749096 │   -73.975398 │
│  17360 │ INVESCO S&P SMCP LOW VOL ETF │ 60515      │ DOWNERS GROVE │    41.822742 │   -88.031668 │
│  17682 │ ALPS ETF VLCTYSHRS EMG MKTS  │ 80203      │ DENVER        │    39.736700 │  -104.987340 │
│  18988 │ CI MRNGSTR US MO INDEX ETF   │ M5J 0A3    │ TORONTO       │    43.653524 │   -79.383907 │
│  19008 │ FIDELITY MSCI INFO TECH ETF  │ 94105-2618 │ SAN FRANCISCO │    37.789152 │  -122

In [ ]:
"""
In this Cell we want to preprocess the names of the compustat datasets
We replace longer terms with their abbreviations or special Characters, to reduce their weight in the name, but keeping them, to improve matching score
Also we want to get rid of whitespaces and dashes
"""

query_str = """
UPDATE compustat
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)


-- END OF COMPUSTAT PREPROCESSING --


-- START OF NETS PREPROCESSING --


In [ ]:
con.execute('DROP TABLE IF EXISTS nets_all')
con.execute('CREATE TABLE nets_all AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_all_data')

In [ ]:
"""
In this Cell we want to preprocess the names of the Nets datasets
We replace longer terms with their abbreviations or special Characters, to reduce their weight in the name, but keeping them, to improve matching score
Also we want to get rid of whitespaces and dashes
"""

query_str = """
UPDATE nets_all
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [ ]:
"""
In this Cell we will fix the longitudes of the NETS all data
The coordinates are not correct in our NETS sample. 
Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
We need to invert the longitudes
"""
sql_query = """
    UPDATE nets_all
    SET longitude = -longitude;
"""
con.execute(sql_query)

-- NETS PUB PREPROCESSING --

In [22]:
con.execute('DROP TABLE IF EXISTS nets_pub')
con.execute('CREATE TABLE nets_pub AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_pub_data')

In [23]:
"""
In this Cell we want to preprocess the names of the NETS datasets
"""
query_str = """
UPDATE nets_pub
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [24]:
"""
In this Cell we will fix the longitudes of the NETS pub data
The coordinates are not correct in our NETS sample. 
Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
We need to invert the longitudes
"""

sql_query = """
    UPDATE nets_pub
    SET longitude = -longitude;
"""
con.execute(sql_query)

In [ ]:
con.commit()
con.close()